In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import os
import warnings

In [62]:
# Reading the CEMS CSV file
AR_CEMS = pd.read_csv("TESTING.csv")

# Reading the CEMS CSV file
epa_crosswalk = pd.read_csv("EPA_crosswalk.csv")

# Convert the 'date' column to datetime data type
AR_CEMS['date'] = pd.to_datetime(AR_CEMS['date'])

/var/folders/x6/sgl68t514w936qf5pph09n040000gn/T/ipykernel_2762/4175471011.py:2: DtypeWarning: Columns (3,25,27,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  AR_CEMS = pd.read_csv("TESTING.csv")


In [63]:
generators = []

for i in range(11):
    generator = pd.read_csv("Generators_Cleaned/" + str(2010 + i) + ".csv")
    generators.append(generator)
    
generators[0].head()

,Utility Name,Plant Code,Plant Name,State,County,Generator ID,Nameplate,Summer Capability,Winter Capability,Energy Source 1
0,Alabama Power Co,3,Barry,AL,Mobile,1,153.1,138.0,138.0,BIT
1,Alabama Power Co,3,Barry,AL,Mobile,2,153.1,137.0,137.0,BIT
2,Alabama Power Co,3,Barry,AL,Mobile,3,272.0,249.0,249.0,BIT
3,Alabama Power Co,3,Barry,AL,Mobile,4,403.7,362.0,362.0,BIT
4,Alabama Power Co,3,Barry,AL,Mobile,5,788.8,726.5,726.5,BIT


In [64]:
# Converting 'EIA_PLANT_ID' to int64
epa_crosswalk['EIA_PLANT_ID'] = pd.to_numeric(epa_crosswalk['EIA_PLANT_ID'], errors='coerce').astype('Int64')
epa_crosswalk.dropna(subset=['EIA_PLANT_ID'], inplace=True)
epa_crosswalk['EIA_PLANT_ID'] = pd.to_numeric(epa_crosswalk['EIA_PLANT_ID'], errors='coerce').astype('int64')

# Converting 'CAMD_UNIT_ID' and 'EIA_GENERATOR_ID' to <class 'str'>
epa_crosswalk['CAMD_UNIT_ID'] = epa_crosswalk['CAMD_UNIT_ID'].astype(str)
epa_crosswalk['EIA_GENERATOR_ID'] = epa_crosswalk['EIA_GENERATOR_ID'].astype(str)
AR_CEMS['unitId'] = AR_CEMS['unitId'].astype(str)


# Merging EPA Crosswalk and CEMS
AR_CEMS = pd.merge( AR_CEMS, 
                    epa_crosswalk, 
                    left_on=['facilityId', 'unitId'], 
                    right_on=['CAMD_PLANT_ID', 'CAMD_UNIT_ID'], 
                    how='left')


# Dropping duplicate columns
AR_CEMS.drop(columns=['facilityId', 'unitId','CAMD_PLANT_ID', 'CAMD_UNIT_ID'], inplace=True)


# Bringing 'EIA_PLANT_ID' and 'EIA_GENERATOR_ID' to the front
AR_CEMS.insert(2, 'EIA_PLANT_ID', AR_CEMS.pop('EIA_PLANT_ID'))
AR_CEMS.insert(3, 'EIA_GENERATOR_ID', AR_CEMS.pop('EIA_GENERATOR_ID'))

# print(AR_CEMS.columns.to_list())

In [65]:
columns_to_keep = ['stateCode', 'facilityName', 'EIA_PLANT_ID', 'EIA_GENERATOR_ID', 
                   'date', 'hour','grossLoad', 'heatInput']

AR_CEMS = AR_CEMS.filter(items=columns_to_keep)

AR_CEMS.rename(columns={
    'stateCode': 'State Code',
    'facilityName': 'Plant Name',
    'EIA_PLANT_ID': 'Plant Code',
    'EIA_GENERATOR_ID': 'Generator ID',
    'date': 'Date',
    'hour': 'Hour',
    'grossLoad': 'Gross Load',
    'heatInput': 'Heat Input'
}, inplace=True)

AR_CEMS.head()

,State Code,Plant Name,Plant Code,Generator ID,Date,Hour,Gross Load,Heat Input
0,AR,Thomas Fitzhugh,201,1,2010-01-01,0,98.0,877.2
1,AR,Thomas Fitzhugh,201,2,2010-01-01,0,98.0,877.2
2,AR,Thomas Fitzhugh,201,1,2010-01-01,1,97.0,876.5
3,AR,Thomas Fitzhugh,201,2,2010-01-01,1,97.0,876.5
4,AR,Thomas Fitzhugh,201,1,2010-01-01,2,98.0,879.7


In [75]:
merged_table_list = []

for i in range(11):
    AR_filtered = AR_CEMS[AR_CEMS['Date'].dt.year == (2010 + i)]

    merged_table = pd.merge(AR_filtered, 
                            generators[i], 
                            left_on=['Plant Code', 'Generator ID'], 
                            right_on=['Plant Code', 'Generator ID'], how='left')

    merged_table.drop(columns=['Plant Name_y', 'State', 'Utility Name'], inplace=True)

    merged_table.rename(columns={'Plant Name_x': 'Plant Name'}, inplace=True)
    
    merged_table_list.append(merged_table)

merged_table_list[0]

,State Code,Plant Name,Plant Code,Generator ID,Date,Hour,Gross Load,Heat Input,County,Nameplate,Summer Capability,Winter Capability,Energy Source 1
0,AR,Thomas Fitzhugh,201,1,2010-01-01,0,98.0,877.2,Franklin,59.0,62.0,62.0,NG
1,AR,Thomas Fitzhugh,201,2,2010-01-01,0,98.0,877.2,Franklin,126.0,103.0,103.0,NG
2,AR,Thomas Fitzhugh,201,1,2010-01-01,1,97.0,876.5,Franklin,59.0,62.0,62.0,NG
3,AR,Thomas Fitzhugh,201,2,2010-01-01,1,97.0,876.5,Franklin,126.0,103.0,103.0,NG
4,AR,Thomas Fitzhugh,201,1,2010-01-01,2,98.0,879.7,Franklin,59.0,62.0,62.0,NG
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112483,AR,Union Power Station,55380,STG1,2010-12-18,17,210.0,1729.3,Union,255.0,215.0,232.0,NG
112484,AR,Union Power Station,55380,CTG2,2010-12-18,18,237.0,1917.9,Union,176.0,145.0,152.0,NG
112485,AR,Union Power Station,55380,STG1,2010-12-18,18,237.0,1917.9,Union,255.0,215.0,232.0,NG
112486,AR,Union Power Station,55380,CTG2,2010-12-18,19,258.0,2231.5,Union,176.0,145.0,152.0,NG
